In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import librosa

warnings.filterwarnings("ignore")

In [30]:
audio_dir = '../backend/data'
n_mfcc = 20
frame_size = 0.025
frame_stride = 0.01
num_classes = 2
sample_rate = 44100
batch_size=8
epochs=10
audio_files = []
for root, dirs, files in os.walk(audio_dir):
    for file in files:
        audio_files.append(os.path.join(root,file))

X = []
y = []

print(audio_files)

['../backend/data\\blues\\blues.00000.wav', '../backend/data\\blues\\blues.00001.wav', '../backend/data\\blues\\blues.00002.wav', '../backend/data\\blues\\blues.00003.wav', '../backend/data\\blues\\blues.00004.wav', '../backend/data\\blues\\blues.00005.wav', '../backend/data\\blues\\blues.00006.wav', '../backend/data\\blues\\blues.00007.wav', '../backend/data\\blues\\blues.00008.wav', '../backend/data\\blues\\blues.00009.wav', '../backend/data\\blues\\blues.00010.wav', '../backend/data\\blues\\blues.00011.wav', '../backend/data\\blues\\blues.00012.wav', '../backend/data\\blues\\blues.00013.wav', '../backend/data\\blues\\blues.00014.wav', '../backend/data\\blues\\blues.00015.wav', '../backend/data\\blues\\blues.00016.wav', '../backend/data\\blues\\blues.00017.wav', '../backend/data\\blues\\blues.00018.wav', '../backend/data\\blues\\blues.00019.wav', '../backend/data\\blues\\blues.00020.wav', '../backend/data\\blues\\blues.00021.wav', '../backend/data\\blues\\blues.00022.wav', '../backen

In [ ]:
import os
import librosa
import numpy as np
from pydub import AudioSegment
import tempfile

audio_dir = 'data'
n_mfcc = 20
frame_size = 0.025
frame_stride = 0.01
sample_rate = 44100
max_frames = 1300  # truncate all MFCCs to this number of frames

X = []
y = []

# Automatically assign numeric labels based on folder names
genres = sorted([d for d in os.listdir(audio_dir) if os.path.isdir(os.path.join(audio_dir, d))])
genre_to_label = {genre: idx for idx, genre in enumerate(genres)}

def truncate_mfcc(mfcc, max_len):
    """Truncate MFCC to max_len frames"""
    if mfcc.shape[1] > max_len:
        return mfcc[:, :max_len]
    else:
        return mfcc

for genre in genres:
    genre_path = os.path.join(audio_dir, genre)
    for file in os.listdir(genre_path):
        if file.lower().endswith('.wav'):
            file_path = os.path.join(genre_path, file)
            try:
                # Try loading normally
                audio, sr = librosa.load(file_path, sr=sample_rate, mono=True)
            except Exception:
                # Convert to 16-bit PCM WAV if needed
                try:
                    with tempfile.NamedTemporaryFile(suffix=".wav") as tmpfile:
                        audio_segment = AudioSegment.from_file(file_path)
                        audio_segment = audio_segment.set_channels(1) \
                                                     .set_frame_rate(sample_rate) \
                                                     .set_sample_width(2)
                        audio_segment.export(tmpfile.name, format="wav")
                        audio, sr = librosa.load(tmpfile.name, sr=sample_rate, mono=True)
                except Exception as e:
                    print(f"Skipping {file_path}: {e}")
                    continue

            # Extract MFCC
            mfcc = librosa.feature.mfcc(
                y=audio,
                sr=sr,
                n_mfcc=n_mfcc,
                n_fft=int(frame_size * sr),
                hop_length=int(frame_stride * sr)
            )

            # Truncate MFCC to fixed number of frames
            mfcc = truncate_mfcc(mfcc, max_frames)

            X.append(mfcc.T)  # shape: (frames, n_mfcc)
            y.append(genre_to_label[genre])

# Convert to NumPy arrays
X = np.array(X)  # shape: (num_samples, max_frames, n_mfcc)
y = np.array(y)

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Genres mapping:", genre_to_label)


Skipping ../backend/data\jazz\jazz.00054.wav: [WinError 2] The system cannot find the file specified
Features shape: (999, 1300, 20)
Labels shape: (999,)
Genres mapping: {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
